In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import pandas_profiling
try: 
    from iterstrat.ml_stratifiers import MultilabelStratifiedShuffleSplit
except Exception:
    !pip install ../input/iterative-stratification/iterative-stratification-master
df = pd.read_csv("../input/clickstream-data-for-online-shopping/e-shop clothing 2008.csv", delimiter=";")
df

In [ ]:
df[(df["day"] == 1) & (df.month == 4)]

In [ ]:
# !pip install ../input/iterative-stratification/iterative-stratification-master

In [ ]:
df = df.groupby(["year", "month", "day"])["session ID"].nunique().reset_index()
df

In [ ]:
df["month"] = df["month"]
df

In [ ]:
df["date"] = df.day.astype(str) + "-" + (df.month).astype(str) + "-"+ df.year.astype(str)
df["date"] = pd.to_datetime(df["date"], format="%d-%m-%Y")
df

In [ ]:
df["season"] = df["month"] % 12 // 3 + 1
df["quarter"] = pd.DatetimeIndex(df.date).quarter
month_in_year = 12
import numpy as np
df['month_sin'] = np.sin(2*np.pi*df.month/month_in_year).astype(np.float16)
df['month_cos'] = np.cos(2*np.pi*df.month/month_in_year).astype(np.float16)
quarters_in_year = 4
df['quarter_sin'] = np.sin(2*np.pi*df.quarter/quarters_in_year).astype(np.float16)
df['quarter_cos'] = np.cos(2*np.pi*df.quarter/quarters_in_year).astype(np.float16)
seasons_in_year = 4
df['season_sin'] = np.sin(2*np.pi*df.season/seasons_in_year).astype(np.float16)
df['season_cos'] = np.cos(2*np.pi*df.season/seasons_in_year).astype(np.float16)
day_in_month = 30.4375
df['day_sin'] = np.sin(2*np.pi*df.day/day_in_month).astype(np.float16)
df['day_cos'] = np.cos(2*np.pi*df.day/day_in_month).astype(np.float16)
df


In [ ]:
dates = df[["year", "month", "day"]]
df = df.drop(["year", "month", "day", "date", "season", "quarter"], axis=1)
df

In [ ]:
RANK_GAUSS = True
if RANK_GAUSS:
    import cupy as cp
    from cupyx.scipy.special import erfinv
    import cudf as gd

    import numpy as np
    import pandas as pd
    from scipy.special import erfinv as sp_erfinv

    x_gpu = cp.asarray(df["session ID"].values) # input array

    r_gpu = x_gpu.argsort().argsort() # compute the rank

    r_gpu = (r_gpu/r_gpu.max()-0.5)*2 # scale to (-1,1)
    epsilon = 1e-6
    r_gpu = cp.clip(r_gpu,-1+epsilon,1-epsilon)

    r_gpu = erfinv(r_gpu) # map to gaussian
    df["session ID"] = cp.asnumpy(r_gpu)
else:
    from sklearn.preprocessing import StandardScaler, Normalizer, MinMaxScaler
    scaler = StandardScaler()
    df["session ID"] = scaler.fit_transform(df["session ID"].values.reshape(-1, 1))
df

In [ ]:
from cuml.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 10))
knn = NearestNeighbors(n_neighbors=df.shape[1])
knn.fit(df.values)
distances, indices = knn.kneighbors(df.values)
distances = np.sort(distances, axis=0)
distances = distances[:,1]
plt.plot(distances)

In [ ]:
from cuml.cluster import DBSCAN
import matplotlib.pyplot as plt

for x in np.arange(0, 2, 0.05):
    d = DBSCAN(eps=x.astype(np.float16), min_samples=df.shape[1])
    d.fit(df.values.astype(np.float32))
    train_cluster = d.labels_
    plt.figure()
    sns.countplot(x=train_cluster)
    plt.title(f"{x}")

In [ ]:
df

In [ ]:
df.to_csv("DBSCAN_Clickthrough.csv", index=False)

In [ ]:
d = DBSCAN(eps=1.15, min_samples=df.shape[1])
d.fit(df.values.astype(np.float32))
train_cluster = d.labels_
df["DBSCAN_cluster"] = d.labels_
df = pd.concat([df, dates], axis=1)
df

In [ ]:
sns.countplot(df["DBSCAN_cluster"])

In [ ]:
df.to_csv("DBSCAN_Clickthrough_ExpectedResult.csv", index=False)

In [ ]:
df_fp = pd.read_csv("../input/clickstream-data-for-online-shopping/e-shop clothing 2008.csv", delimiter=";")
gr_df = pd.get_dummies(df_fp[["page 2 (clothing model)", "session ID"]], columns=["page 2 (clothing model)"], prefix="", prefix_sep='').groupby("session ID").sum().astype(bool).reset_index()
gr_df

In [ ]:
df[df["DBSCAN_cluster"] == 0].head()

In [ ]:
cluster=df_fp.merge(df[df["DBSCAN_cluster"] == 0], how="inner", on=["month", "day"])
cluster = cluster[["session ID_x", "page 2 (clothing model)"]]
cluster

In [ ]:
cluster_fp = pd.get_dummies(cluster[["page 2 (clothing model)", "session ID_x"]], columns=["page 2 (clothing model)"], prefix="", prefix_sep='').groupby("session ID_x").sum().astype(bool).reset_index()
cluster_fp

In [ ]:
cluster_fp[cluster_fp.columns[1:]].sum()

In [ ]:
from iterstrat.ml_stratifiers import MultilabelStratifiedShuffleSplit

msss = MultilabelStratifiedShuffleSplit(n_splits = 1, test_size=0.2, random_state=42)
idx = msss.split(cluster_fp, cluster_fp[cluster_fp.columns[1:]])
tr, te = next(idx)
train, test = cluster_fp.loc[tr].drop("session ID_x", axis=1), cluster_fp.loc[te].drop("session ID_x", axis=1)
train

In [ ]:
test

In [ ]:
!pip install csv2arff

train.to_csv("train_cluster0.csv", index=False)
test.to_csv("test_cluster0.csv", index=False)
!csv2arff train_cluster0.csv train_cluster0.arff
!csv2arff test_cluster0.csv test_cluster0.arff

In [ ]:
from iterstrat.ml_stratifiers import MultilabelStratifiedShuffleSplit

msss = MultilabelStratifiedShuffleSplit(n_splits = 1, test_size=0.2, random_state=42)
idx = msss.split(gr_df, gr_df[gr_df.columns[1:]])
tr, te = next(idx)
train, test = gr_df.loc[tr], gr_df.loc[te]
train

In [ ]:
from iterstrat.ml_stratifiers import MultilabelStratifiedShuffleSplit

msss = MultilabelStratifiedShuffleSplit(n_splits = 1, test_size=0.2, random_state=42)
idx = msss.split(gr_df, gr_df[gr_df.columns[1:]])
tr, te = next(idx)
train, test = gr_df.loc[tr], gr_df.loc[te]
train

In [ ]:
test

In [ ]:
# !pip install csv2arff

# train.to_csv("train_noID.csv", index=False)
# test.to_csv("test_noID.csv", index=False)
# !csv2arff train_noID.csv train_noID.arff
# !csv2arff test_noID.csv test_noID.arff

https://stackoverflow.com/a/54115075/11235205

In [ ]:
# profile = pandas_profiling.ProfileReport(df)
# profile.to_file("ClickstreamPandasProfiling.html")